# <center> OBTENER LISTA CON LOS NOMBRES DE TODAS LAS RAMAS LOCALES DE UN REPOSITORIO GIT</center>

In [1]:
def obtener_lista_nombres_ramas_git():

    stdout = subprocess.check_output('git branch'.split())
    out = stdout.decode()
    branches = [b.strip('* ') for b in out.splitlines()]
    return branches

# <center> EJECUTAR COMANDOS SHELL DESDE FUNCION PYTHON</center>

In [2]:
import subprocess

def ejecutar_comando_shell(comando='git branch', response=False):

    resultado_comando_ejecutado = subprocess.check_output(comando, shell=True, text=True)
    print('############################################# COMANDO A EJECUTAR #############################################')
    print(comando)
    print('####################################### RESULTADO DE COMANDO EJECUTADO #######################################')
    #para evitar el salto de linea usa el parametro end=''
    print(resultado_comando_ejecutado, end='')
    print('##############################################################################################################')
    print()
    if response:
        return resultado_comando_ejecutado

# <center> FUNCION LAMBDA PARA EJECUTAR COMANDOS GIT</center>

In [3]:
ejecutar_comando_git = lambda comando : ejecutar_comando_shell(comando)

# <center> CREAR ARCHIVOS</center>

In [4]:
import time
from datetime import datetime

def crear_fecha():
    
    import time
    from datetime import datetime

    # detiene el tiempo para que no todos los archivos generados queden con la misma fecha de timestamp
    time.sleep(0.1)
        
    # Getting the current date and time
    dt = datetime.now()

    # getting the timestamp
    ts = datetime.timestamp(dt)

    return ts

def generar_archivos(todos_los_archivos=True):

    lista_de_archivos = ['info.txt','Dockerfile','Manifest.yaml','imagen.png','main.py','connection.java','calculos.c','addon.php','utils.gif','query.sql','dataset.avro','link.vs']
    
    lista_de_archivos = lista_de_archivos if todos_los_archivos else [lista_de_archivos[0]]
    
    print('lista_de_archivos = {lista_de_archivos}'.format(lista_de_archivos=lista_de_archivos))
    
    for archivo in lista_de_archivos:
        comando = 'echo {crear_archivo} >> {archivo}'.format(crear_archivo=crear_fecha(), archivo=archivo)
        ejecutar_comando_shell(comando)

    return lista_de_archivos


def crear_commits(total_commits=5):
    
    import random
    
    for commit_numero in range(total_commits):
        
        numero_aleatorio = random.randint(1, 10_000_000)
        lista_de_archivos = generar_archivos()
        archivo_aleatorio = random.choice(lista_de_archivos)
        commit_indice = commit_numero + 1

        comando = 'git status'
        ejecutar_comando_git(comando)

        comando = 'git add .'
        ejecutar_comando_git(comando)

        comando = 'git status'
        ejecutar_comando_git(comando)

        comando = 'git commit -m "PRUEBA de COMMIT numero {commit_indice} con fecha {fecha} y valor aleatorio {numero_aleatorio} y contiene el archivo {archivo_aleatorio}"'.format(commit_indice=commit_indice, numero_aleatorio=numero_aleatorio, fecha=crear_fecha(), archivo_aleatorio=archivo_aleatorio)
        ejecutar_comando_git(comando)

    comando = 'git log --oneline'
    ejecutar_comando_git(comando)

    comando = 'git status'
    ejecutar_comando_git(comando)

# <center> ELIMINAR ARCHIVOS</center>

In [5]:
def listar_archivos_eliminables_en_directorio():
    import os
    
    listado_de_archivos = os.listdir(directory)
    
    archivos_eliminables = [archivo for archivo in listado_de_archivos if '.git' not in archivo]
    
    return archivos_eliminables
    
def eliminar_directorio(nombre_repositorio):
    
    import shutil
    import os
    import stat
    from os import path
    
    for root, dirs, files in os.walk("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio)):  
        for dir in dirs:
            try:
                os.chmod(path.join(root, dir), stat.S_IRWXU)
            except:
                print('el archivo {archivo} no existe'.format(archivo=archivo))                
        for file in files:
            try:
                os.chmod(path.join(root, file), stat.S_IRWXU)
            except:
                print('el archivo {archivo} no existe'.format(archivo=archivo))
    try: 
        shutil.rmtree("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio))
    except:
        print('el repositorio {nombre_repositorio} no existe'.format(nombre_repositorio=nombre_repositorio))

def eliminar_archivo(archivo):
    from os import remove
    remove(archivo)

def eliminar_archivos_directorio():
    import os
    
    lista_de_archivos_eliminables = listar_archivos_eliminables_en_directorio()
    
    for archivo in lista_de_archivos_eliminables:
            os.remove(archivo)
        
    return 'Eliminacion Completada con Exito!!! de {lista_de_archivos_eliminables}'.format(lista_de_archivos_eliminables=lista_de_archivos_eliminables)


def volver_al_commit_sin_cambios(commit_id):
    print("REGRESANDO AL COMMIT' {commit_id}".format(commit_id=commit_id))

    comando = 'git reset --hard {commit_id}'.format(commit_id=commit_id)
    ejecutar_comando_git(comando)
    
    comando = 'git status'
    ejecutar_comando_git(comando)
    
def eliminar_todos_los_archivos_hasta_el_commit(commit_id):
    print("REGRESANDO AL COMMIT' {commit_id}".format(commit_id=commit_id))

    comando = 'git reset {commit_id}'.format(commit_id=commit_id)
    ejecutar_comando_git(comando)

    print('ELIMINANDO ARCHIVOS DEL STAGE')
    # eliminar los archivos que no estan en el local repo, para evitar errores al cambiar de rama
    comando = 'git rm --cached "*"'
    ejecutar_comando_git(comando)

    print('ELIMINANDO ARCHIVOS DEL WORKING DIRECTORY')
    # elimina los archivos que estan en el working directory
    comando = 'git clean -f'
    ejecutar_comando_git(comando)

    comando = 'git status'
    ejecutar_comando_git(comando)

    comando = 'git log --oneline'
    ejecutar_comando_git(comando)

    ejecutar_comando_shell()

# <center> CREAR ARCHIVOS, COMMITS Y PUBLICAR RAMAS AL REPOSITORIO </center>

In [6]:
def crear_o_actualizar_archivo(actualizaciones=10, archivo='info.txt'):
    for index in range(actualizaciones):
        comando = "echo actualizacion {index} fecha {fecha} >> {archivo}".format(index=index+1, fecha=datetime.timestamp(datetime.now()), archivo=archivo)
        ejecutar_comando_shell(comando)

def validar_si_existe_rama(nombre_rama, local=False):
    
    comando ='git branch -a'
    resultado = ejecutar_comando_shell(comando, response=True)
    
    rama_base = '{nombre_rama}'.format(nombre_rama=nombre_rama) if local else 'origin/{nombre_rama}'.format(nombre_rama=nombre_rama)

    if rama_base in resultado:
        return True
    return False

def elimina_ramas_en_local_o_remoto_si_existen(nombre_rama, rama_local, rama_remota):
    
    # valida si las ramas existen en REMOTO y LOCAL
    rama_remota_existe = validar_si_existe_rama(nombre_rama=nombre_rama)
    rama_local_existe = validar_si_existe_rama(nombre_rama=nombre_rama, local=True)
    
    # verifica si existe la rama la elimina
    if rama_remota and rama_remota_existe:
        # elimina rama REMOTA existente
        comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)

     # verifica si existe la rama la elimina
    if rama_local and rama_local_existe:                                                                                                            
        # elimina la rama
        comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)
        
    # verifica las ramas actuales
    ejecutar_comando_shell()
    
def crear_commit_por_cada_archivo(todos_los_archivos):
    
    import datetime

    fecha_ahora = datetime.datetime.now()
    
    # crea el listado de archivos
    lista_de_archivos = generar_archivos(todos_los_archivos=todos_los_archivos)
    
    # basado en el listado de archivos, adiciona uno a uno y crea un commit por cada uno
    for archivo in lista_de_archivos:

        comando = 'git add {archivo}'.format(archivo=archivo)
        ejecutar_comando_git(comando)
        
        comando = 'git commit -m "ADICIONADO ARCHIVO {archivo} {fecha_ahora}" -m "- El proposito es generar un registro de archivos con su fecha correspondiente, - Se adiciono el archivo {archivo} al commit, - La fecha donde se creao este commit es {fecha_ahora}"'.format(archivo=archivo, fecha_ahora=fecha_ahora)
        ejecutar_comando_git(comando)

    # verifica si todos los archivos estan en los commits creados
    comando = 'git status'
    ejecutar_comando_git(comando)
    
def generar_archivos_y_hacer_commit(nombre_rama, rama_local, rama_remota, todos_los_archivos):
    
    # elimina cambios a la rama remota si los tiene
    comando = 'git restore .'
    ejecutar_comando_git(comando)
    
    # cambiar a la rama remota, 
    comando = 'git checkout {nombre_rama_principal} --force'.format(nombre_rama_principal=nombre_rama_principal)
    ejecutar_comando_git(comando)
    
    # valida si las ramas existen en REMOTO y LOCAL y las elimina
    elimina_ramas_en_local_o_remoto_si_existen(nombre_rama=nombre_rama, rama_local=rama_local, rama_remota=rama_remota)
    
    if rama_local:
        # crea la rama local
        comando = 'git checkout -b {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_shell(comando)
    
    # adiciona archivos de 1 en 1 y genera 1 commit por cada archivo
    crear_commit_por_cada_archivo(todos_los_archivos=todos_los_archivos)

    # mostrar el log de las ramas en forma resumida y con diagrama de uniones como grafo 
    comando = 'git log --graph --oneline'
    ejecutar_comando_git(comando)

    if rama_remota:
        # crea la rama en remoto
        comando = 'git push --set-upstream origin {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)
    
    ejecutar_comando_shell()

def crear_rama_local_y_remota(nombre_rama, rama_local=True, rama_remota=True, todos_los_archivos=True):
    generar_archivos_y_hacer_commit(nombre_rama=nombre_rama, rama_local=rama_local, rama_remota=rama_remota, todos_los_archivos=todos_los_archivos)
    

def eliminar_rama_local_y_remota(nombre_rama, rama_local=True, rama_remota=True):

    comando = 'git checkout {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
    ejecutar_comando_git(comando)

    if rama_local:
        comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)
    
    if rama_remota:
        comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)

    comando = 'git branch -a'
    ejecutar_comando_git(comando)

# <center> FLUJO DE INFORMACION ENTRE RAMAS DE UN REPOSITORIO GIT</center>
## <center> BRANCH STRATREGY</center>

<center>
    <img width="100%" align="center" src="git_flow.png">
</center>

# <center> CICLO DE VIDA DE UN REPOSITORIO GIT</center>

<center>
    <img width="100%" align="center" src="git_life_cycle.png">
</center>

# <center> FLUJO DE ARCHIVOS DE UN REPOSITORIO GIT</center>

<center>
    <img width="100%" align="center" src="gitflow.png">
</center>

# <center> CONSULTAR LA VERSION DE GIT</center>

In [7]:
comando = 'git --version'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git --version
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
git version 2.25.0.windows.1
##############################################################################################################



# <center> CLONAR UN REPOSITORIO GIT</center>

In [8]:
repositorio_base = 'https://github.com/JorgeCardona'
nombre_del_repositorio = 'comandos_git'

eliminar_directorio(nombre_repositorio=nombre_del_repositorio)

comando = 'git clone {repositorio_base}/{nombre_del_repositorio}.git'.format(repositorio_base=repositorio_base, nombre_del_repositorio=nombre_del_repositorio)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git clone https://github.com/JorgeCardona/comandos_git.git
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



# <center> PROTEGER RAMAS DE UN REPOSITORIO CONTRA ESCRITURA</center>

# <center> CAMBIAR A LA CARPETA PARA TRABAJAR CON EL REPOSITORIO</center>

In [9]:
#importing the os module
import os

#to get the current working directory
directory = os.getcwd()
directory

'C:\\z Github\\skills\\comandos GIT'

In [10]:
os.chdir('{directory}/{repository}'.format(directory=directory, repository=nombre_del_repositorio))
directory = os.getcwd()
directory

'C:\\z Github\\skills\\comandos GIT\\comandos_git'

# <center> MOSTRAR INFORMACION DEL REPOSITORIO</center>

In [11]:
comando = 'git remote show origin'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git remote show origin
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* remote origin
  Fetch URL: https://github.com/JorgeCardona/comandos_git.git
  Push  URL: https://github.com/JorgeCardona/comandos_git.git
  HEAD branch: main
  Remote branches:
    bisect_branch tracked
    main          tracked
    multi_commits tracked
  Local branch configured for 'git pull':
    main merges with remote main
  Local ref configured for 'git push':
    main pushes to main (up to date)
##############################################################################################################



# <center> LISTAR LAS RAMAS DE UN REPOSITORIO </center>

### LISTA TODAS LAS RAMAS LOCALES Y LA RAMA PRINCIPAL DEL REPOSITORIO

In [12]:
comando = 'git branch'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
##############################################################################################################



### LISTA TODAS LAS RAMAS REMOTAS DEL REPOSITORIO

In [13]:
comando = 'git branch -r'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch -r
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  origin/HEAD -> origin/main
  origin/bisect_branch
  origin/main
  origin/multi_commits
##############################################################################################################



### LISTA TODAS LAS RAMAS LOCALES INCLUYENDO LAS RAMAS OCULTAS

In [14]:
comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/bisect_branch
  remotes/origin/main
  remotes/origin/multi_commits
##############################################################################################################



# <center> CREAR UNA RAMA BASADA EN LA RAMA ACTUAL DE UN REPOSITORIO</center>

In [15]:
nombre_rama_principal = 'main'
nombre_rama_a_crear = 'nueva_rama'
comando = f'git branch {nombre_rama_a_crear}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git branch nueva_rama
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  nueva_rama
##############################################################################################################



# <center> CAMBIAR A OTRA RAMA DEL REPOSITORIO</center>

### GIT CHECKOUT

In [16]:
comando = f'git checkout {nombre_rama_a_crear}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout nueva_rama
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* nueva_rama
##############################################################################################################



### GIT SWITCH

In [17]:
comando = 'git switch {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

comando = f'git switch {nombre_rama_a_crear}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git switch main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  nueva_rama
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git switch nueva_rama
####################################### RESULTADO DE COMANDO EJECUTADO ###################################

# <center> CREAR UNA RAMA BASADA EN LA ACTUAL Y CAMBIA A LA RAMA CREADA AL MISMO TIEMPO</center>

### GIT CHECKOUT

In [18]:
nombre_rama_checkout = 'rama_acceso_rapido_checkout'
comando = f'git checkout -b {nombre_rama_checkout}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -b rama_acceso_rapido_checkout
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  nueva_rama
* rama_acceso_rapido_checkout
##############################################################################################################



### GIT SWITCH

In [19]:
nombre_rama_switch = 'rama_acceso_rapido_switch'
comando = f'git switch -c {nombre_rama_switch}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git switch -c rama_acceso_rapido_switch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  nueva_rama
  rama_acceso_rapido_checkout
* rama_acceso_rapido_switch
##############################################################################################################



# <center> CREAR NUEVA RAMA LOCAL DE UNA RAMA REMOTA</center>

### GIT CHECKOUT

In [20]:
nombre_rama_remota = 'multi_commits'
nombre_rama_remota_copia_checkout = f'{nombre_rama_remota}_copia_checkout'
comando = 'git checkout -b {copia_rama_remota} origin/{nombre_rama_remota}'.format(copia_rama_remota=nombre_rama_remota_copia_checkout, nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -b multi_commits_copia_checkout origin/multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits_copia_checkout' set up to track remote branch 'multi_commits' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* multi_commits_copia_checkout
  nueva_rama
  rama_acceso_rapido_checkout
  rama_acceso_rapido_switch
##############################################################################################################



### GIT SWITCH

In [21]:
nombre_rama_remota_copia_switch = f'{nombre_rama_remota}_copia_switch'
comando = 'git switch -c {copia_rama_remota} origin/{nombre_rama_remota}'.format(copia_rama_remota=nombre_rama_remota_copia_switch, nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git switch -c multi_commits_copia_switch origin/multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits_copia_switch' set up to track remote branch 'multi_commits' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  multi_commits_copia_checkout
* multi_commits_copia_switch
  nueva_rama
  rama_acceso_rapido_checkout
  rama_acceso_rapido_switch
##############################################################################################################



# <center> DESCARGAR RAMA REMOTA A LOCAL Y CAMBIAR A LA RAMA BAJADA AL MISMO TIEMPO</center>

In [22]:
comando = 'git checkout -t origin/{nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -t origin/multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits' set up to track remote branch 'multi_commits' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* multi_commits
  multi_commits_copia_checkout
  multi_commits_copia_switch
  nueva_rama
  rama_acceso_rapido_checkout
  rama_acceso_rapido_switch
##############################################################################################################



# <center> RENOMBRAR UNA RAMA DEL REPOSITORIO</center>

In [23]:
ejecutar_comando_shell()

comando = 'git checkout {nombre_rama_remota_copia_checkout}'.format(nombre_rama_remota_copia_checkout=nombre_rama_remota_copia_checkout)
ejecutar_comando_git(comando)

nuevo_nombre_rama_renombrada_checkout = f'{nombre_rama_a_crear}_renombrada_checkout'
comando = f'git branch -m {nombre_rama_a_crear} {nuevo_nombre_rama_renombrada_checkout}'
ejecutar_comando_git(comando)

# Alternativa
nuevo_nombre_rama_acceso_rapido_renombrada_checkout = f'{nombre_rama_checkout}_renombrada_checkout'
comando = f'git branch --move {nombre_rama_checkout} {nuevo_nombre_rama_acceso_rapido_renombrada_checkout}'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* multi_commits
  multi_commits_copia_checkout
  multi_commits_copia_switch
  nueva_rama
  rama_acceso_rapido_checkout
  rama_acceso_rapido_switch
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout multi_commits_copia_checkout
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/multi_commits'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #####

# <center> CREAR RAMA VACIA QUE NO ESTE RELACIONADA CON NINGUN CONTENIDO DE OTRA RAMA</center>
<center>
    <img width="100%" align="center" src="ciclo_de_vida_archuvo_git.png">
</center>

### UNTRACKED - ES UN FICHERO NUEVO QUE SE HA CREADO Y GIT TODAVIA NO TIENE CONSTANCIA, NO HACE PARTE DE NINGUN COMMIT ANTERIOR
### TRACKED - UNSTAGE -> FICHERO QUE HACE PARTE DE COMMITS ANTERIORES, HA SIDO MODIFICADO PERO TODAVIA NO SE HA ENVIADO AL AREA DE STAGING
### TRACKED - STAGE -> SE REALIZO UN CAMBIO Y SE HA ENVIADO A LA ZONA STAGING (git add archivo)
### TRACKED - COMMITED -> UN ARCHIVO QUE SE HA HECHO COMMIT Y QUE YA ESTA EN LA ZONA EL REPOSITORIO LOCAL (git commit -m 'archivo adicionado al repo local')

### CREA LA RAMA

In [24]:
nombre_rama = 'utils'
comando = 'git switch --orphan {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git switch --orphan utils
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  multi_commits
  multi_commits_copia_checkout
  multi_commits_copia_switch
  nueva_rama_renombrada_checkout
  rama_acceso_rapido_checkout_renombrada_checkout
  rama_acceso_rapido_switch
  remotes/origin/HEAD -> origin/main
  remotes/origin/bisect_branch
  remotes/origin/main
  remotes/origin/multi_commits
#########################################################################################

### CREA ARCHIVOS EN EL WORKING DIRECTORY

In [25]:
generar_archivos()

comando = 'git status'
ejecutar_comando_git(comando)

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1680998476.87604 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1680998476.991095 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR ########

## <center> VALIDAR ARCHIVOS EN EL WORKING DIRECTORY

In [26]:
comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	info.txt
	link.vs
	main.py
	query.sql
	utils.gif

nothing added to commit but untracked files present (use "git add" to track)
##############################################################################################################



## <center> ADICIONA LOS ARCHIVOS AL STAGE

#### ADICIONA UN SOLO ARCHIVO AL STAGE

In [27]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   info.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	link.vs
	main.py
	query.sql
	utils.gif

########################################################################

#### ADICIONA TODOS LOS ARCHIVOS AL STAGE

In [28]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Dockerfile
	new file:   Manifest.yaml
	new file:   addon.php
	new file:   calculos.c
	new file:   connection.java
	new file:   dataset.avro
	new file:   imagen.png
	new file:   info.txt
	new file:   link.vs
	new file:   main.py
	new file:   query.sql
	new file:   utils.gif

##############################

## <center> ELIMINA LOS ARCHIVOS DEL STAGE Y LOS RETORNA AL WORKING DIRECTORY

#### ELIMINA UNO DE LOS ARCHIVOS DEL STAGE

In [29]:
comando = 'git rm --cached info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git rm --cached info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
rm 'info.txt'
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Dockerfile
	new file:   Manifest.yaml
	new file:   addon.php
	new file:   calculos.c
	new file:   connection.java
	new file:   dataset.avro
	new file:   imagen.png
	new file:   link.vs
	new file:   main.py
	new file:   query.sql
	new file:   utils.gif

Untracked files:
  (use

#### ELIMINA TODOS DE LOS ARCHIVOS DEL STAGE

In [30]:
comando = 'git rm --cached "*"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git rm --cached "*"
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
rm 'Dockerfile'
rm 'Manifest.yaml'
rm 'addon.php'
rm 'calculos.c'
rm 'connection.java'
rm 'dataset.avro'
rm 'imagen.png'
rm 'link.vs'
rm 'main.py'
rm 'query.sql'
rm 'utils.gif'
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	info.txt
	link.vs
	main.py


## <center>ARCHIVOS EN EL REPOSITORIO LOCAL - COMMIT - CONFIRMACION

## ELIMINA TODOS LOS ARCHIVOS UNTRACKED DEL WORKING DIRECTORY

In [31]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   info.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	link.vs
	main.py
	query.sql
	utils.gif

########################################################################

### CREA UN COMMIT SOLO CON EL SUBJECT

In [32]:
comando = 'git commit -m "adicionado archivo INFO.txt"'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git commit -m "adicionado archivo INFO.txt"
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[utils (root-commit) 4700b6c] adicionado archivo INFO.txt
 1 file changed, 1 insertion(+)
 create mode 100644 info.txt
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 4700b6c5c90ca456f52b761f30c60994ecb6fbb0
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Apr 8 19:01:24 2023 -0500

    adicionado archivo INFO.txt
#####################################################################################################

### ADICIONA ARCHIVOS Y CREA UN COMMIT CON EL SUBJECT Y EL BODY

In [34]:
comando = 'git add connection.java Manifest.yaml'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionados archivos connection.java y Manifest.yaml" -m "los archivos de este commit ayudan con la configuracion de la aplicacion...." '
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add connection.java Manifest.yaml
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git commit -m "adicionados archivos connection.java y Manifest.yaml" -m "los archivos de este commit ayudan con la configuracion de la aplicacion...." 
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[utils 08c5ec4] adicionados archivos connection.java y Manifest.yaml
 2 files changed, 2 insertions(+)
 create mode 100644 Manifest.yaml
 create mode 100644 connection.java
#################################################################################

## SE PUEDE ADICIONAR UN MENSAJE DESDE UN EDITOR DE TEXTO

In [36]:
comando = 'git add main.py'
ejecutar_comando_git(comando)

comando = 'git commit'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add main.py
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git commit 
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[utils 5fb6b69] TTTT
 1 file changed, 1 insertion(+)
 create mode 100644 main.py
##############################################################################################################



#### ELIMINA TODOS LOS ARCHIVOS UNTRACKED DEL WORKING DIRECTORY

In [ ]:
comando = 'git clean -f'
ejecutar_comando_git(comando)

In [ ]:
comando = 'git status'
ejecutar_comando_git(comando)

<center>
    <img width="100%" height="100%" align="center" src="ciclo_de_vida_archuvo_git.png">
</center>

### UNTRACKED - ES UN FICHERO NUEVO QUE SE HA CREADO Y GIT TODAVIA NO TIENE CONSTANCIA, NO HACE PARTE DE NINGUN COMMIT ANTERIOR
### TRACKED - UNSTAGE -> FICHERO QUE HACE PARTE DE COMMITS ANTERIORES, HA SIDO MODIFICADO PERO TODAVIA NO SE HA ENVIADO AL AREA DE STAGING
### TRACKED - STAGE -> SE REALIZO UN CAMBIO Y SE HA ENVIADO A LA ZONA STAGING (git add archivo)
### TRACKED - COMMITED -> UN ARCHIVO QUE SE HA HECHO COMMIT Y QUE YA ESTA EN LA ZONA EL REPOSITORIO LOCAL (git commit -m 'archivo adicionado al repo local')

### ACTUALIZAR UN ARCHIVO Y VALIDARLO EN LA ZONA DE MODIFIED

In [ ]:
archivo = 'info.txt'
eliminar_archivo(archivo=archivo)
crear_o_actualizar_archivo(actualizaciones=10, archivo=archivo)

In [ ]:
comando = 'git status'
ejecutar_comando_git(comando)

### VERIFICAR QUE MODIFICACIONES INCLUIR BASADO EN EL NIVEL DE PARCHE DEL CODIGO SE ADICIONAN AL AREA DE STAGING - CACHE  O PREPARACION
    y - stage this hunk
    n - do not stage this hunk
    q - quit; do not stage this hunk nor any of the remaining ones
    a - stage this hunk and all later hunks in the file
    d - do not stage this hunk nor any of the later hunks in the file
    g - select a hunk to go to
    / - search for a hunk matching the given regex
    j - leave this hunk undecided, see next undecided hunk
    J - leave this hunk undecided, see next hunk
    k - leave this hunk undecided, see previous undecided hunk
    K - leave this hunk undecided, see previous hunk
    s - split the current hunk into smaller hunks
    e - manually edit the current hunk
    ? - print help

In [ ]:
comando = 'git add -p info.txt'
ejecutar_comando_git(comando)

### ADICIONA LOS ARCHIVOS AL STAGE Y HACE LO ALMACENA EN EL REPOSITORIO LOCAL CON UN COMMIT

In [ ]:
generar_archivos()

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = f'git commit -m "adicionados archivos de prueba {datetime.now()}"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### <center> DESHACER UN COMMIT LOCAL
### ELIMINA LOS ARCHIVOS DEL LOCAL REPOSITORY Y LOS RETORNA AL STAGING AREA

In [ ]:
crear_commits(total_commits=5)
ejecutar_comando_shell()

### ADICIONAR EL HASH DEL COMMIT AL QUE SE QUIERE VOLVER

In [ ]:
# define el commit al que se quiere regresar
commit_id = '71bf9b7' # Dockerfile

comando = 'git reset {commit_id}'.format(commit_id=commit_id)
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

# ELIMINA LOS ARCHIVOS

### LOS ELIMINA DEL STAGE

In [ ]:
# eliminar los archivos que no estan en el local repo, para evitar errores al cambiar de rama
comando = 'git rm --cached "*"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### LOS ELIMINA DEL WORKING DIRECTORY

In [ ]:
# elimina los archivos que estan en el working directory
comando = 'git clean -f'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# RECUPERAR UN ARCHIVO ELIMINADO, LLEVARLO AL STAGE Y LUEGO AL WORKING DIRECTORY

In [ ]:
archivo_a_recuperar = 'Dockerfile'
comando = f'git restore --staged {archivo_a_recuperar}'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = f'git restore {archivo_a_recuperar}'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

## CREA UN NUEVO COMMIT PARA USARLO LUEGO EN EL REFLOG

In [ ]:
generar_archivos(todos_los_archivos=False)

comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = f'git commit -m "adicionado archivo INFO.txt PARA USAR EN REFLOG {datetime.now()}"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> ELIMINAR UNA RAMA DE UN REPOSITORIO GIT</center>

## ELIMINAR LA RAMA

In [ ]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout  {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> PROBAR PUSH A UNA RAMA PROTEGIDA CONTRA ESCRITURA REPOSITORIO GIT</center>

In [ ]:
crear_commits(total_commits=1)
ejecutar_comando_shell()

comando = 'git push origin {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

In [ ]:
commit_id = 'd481740'

volver_al_commit_sin_cambios(commit_id=commit_id)

# <center> OBTENER LA DIFERENCIA EN CONTENIDO ENTRE 2 RAMAS COMMITs DE UN REPOSITORIO</center>

In [ ]:
comando = 'git diff {nombre_rama_principal} {nombre_rama_remota}'.format(nombre_rama_principal=nombre_rama_principal, nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)

# <center> RESTAURAR O RECUPERAR ARCHIVOS MODIFICADOS O ELIMINADOS DE UN REPOSITORIO GIT <br> SOLO MODIFICACIONES EN REPOSITORIO LOCAL, BASADO EN EL ULTIMO COMMIT <br/></center>

In [ ]:
comando = 'git checkout {rama}'.format(rama=nombre_rama_remota_copia_checkout)
ejecutar_comando_git(comando)

generar_archivos()
ejecutar_comando_shell()

In [ ]:
eliminar_archivos_directorio()
ejecutar_comando_shell()

### RESTAURAR UN ARCHIVO ESPECIFICO

In [ ]:
comando = 'git restore README.md'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### RESTAURAR TODOS LOS ARCHIVOS AL MISMO TIEMPO

In [ ]:
comando = 'git restore .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> PUBLICAR UNA RAMA LOCAL QUE NO EXISTE EN REMOTO A UN REPOSITORIO GIT <br>https://github.com/JorgeCardona/comandos_git</br> </center>


In [ ]:
comando = 'git push --set-upstream origin {nombre_rama_remota_copia_checkout}'.format(nombre_rama_remota_copia_checkout=nombre_rama_remota_copia_checkout)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> ADICIONAR ARCHIVOS NUEVOS A LA RAMA</center>

In [ ]:
generar_archivos()

# SUBIR NUEVOS ARCHIVOS A UNA RAMA DEL REPOSITORIO  <br>https://github.com/JorgeCardona/comandos_git</br> 

In [ ]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionados archivos nuevos"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

In [ ]:
# siempre origin para repositorios remotos
comando = 'git push origin'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

## ELIMINAR RAMA DE UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br> 

In [ ]:
comando = 'git push origin -d {nombre_rama_remota_copia_checkout}'.format(nombre_rama_remota_copia_checkout=nombre_rama_remota_copia_checkout)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

### ELIMINAR RAMAS LOCALES QUE NO SE VAN A USAR MAS

In [ ]:
lista_ramas_a_eliminar = [nombre_rama_remota, nombre_rama_switch, nombre_rama_remota_copia_checkout, nombre_rama_remota_copia_switch, nuevo_nombre_rama_renombrada_checkout, nuevo_nombre_rama_acceso_rapido_renombrada_checkout]

comando = 'git checkout  {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)
    
for nombre_rama in lista_ramas_a_eliminar:
    # elimina la rama
    comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)
    
ejecutar_comando_shell()

## FETCH
<center>
    <img width="75%" align="center" src="fecth.gif">
</center>

## PULL
<center>
    <img width="75%" align="center" src="pull.gif">
</center>

# <center> LISTAR EL HISTORIAL DE COMMITS DE UN REPOSITORIO GIT</center>

### VER EL HISTORIAL COMPLETO DE LA RAMA MAIN

In [ ]:
comando = 'git checkout  {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL OTRA RAMA REMOTA

In [ ]:
nombre_rama = 'commit_branch'
crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=True)

In [ ]:
comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL DE LOS ULTIMOS n COMMITS

In [ ]:
comando = 'git log -3' 
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL COMMITS DEL PRIMERO AL ULTIMO

In [ ]:
comando = "git log --reverse --oneline"
ejecutar_comando_git(comando)
ejecutar_comando_shell()

In [ ]:
comando = "git log HEAD~24..HEAD~21 --reverse --oneline"
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### MUESTRA EL HISTORIAL DE COMMITS AGRUPADOS POR AUTOR RAMA ACTUAL

In [ ]:
comando = 'git shortlog --format="%cn on %cd %ae committed %h %s" origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### MUESTRA EL HISTORIAL DE COMMITS AGRUPADOS POR AUTORTODAS LAS RAMAS

In [ ]:
comando = 'git shortlog --all --format="%cn on %cd %ae committed %h %s"'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### MUESTRA EL HISTORIAL DE COMMITS AGRUPADOS POR AUTOR CON METADATOS Y FILTRADO EN EL TIEMPO

In [ ]:
comando = 'git shortlog --format="%s %ae" --since="01-01-2023" --until="31-03-2023" origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### MUESTRA EL HISTORIAL DE COMMITS CON MAS DETALLES POR AUTOR

### REMOTO

In [ ]:
comando = 'git shortlog --format="%cn on %cd %ae committed %h %s" --since="01-01-2023" --until="31-03-2023" origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### LOCAL

In [ ]:
comando = 'git log --pretty=format:"%cn on %cd %ae committed %h %s"  --since="31-03-2023" --until="31-12-2023"'
ejecutar_comando_git(comando)

### FILTRADO POR EMAIL

In [ ]:
comando = 'git shortlog --author="jorgecardona@utp.edu.co" --format="%cn on %cd %ae committed %h %s" --since="01-01-2023" --until="31-03-2023" origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### FILTRADO POR MENSAJE

In [ ]:
comando = 'git shortlog --grep="archivo" --format="%cn on %cd %ae committed %h %s" --since="01-01-2023" --until="31-03-2023" origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### FILTRADO POR ARCHIVO

In [ ]:
comando = 'git log --oneline -- info.txt archivo.pdf Dockerfile origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### FILTRADO POR COMMITS NO MERGED

In [ ]:
comando = 'git log --oneline --no-merges origin/{nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

### VER HISTORIAL RESUMIDO DE UNA RAMA

In [ ]:
comando = 'git log --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL RESUMIDO DE UNA RAMA DE n COMMITS

In [ ]:
comando = 'git log --oneline -3'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## VER HISTORIAL RESUMIDO DE UNA RAMA SIN ESTAR EN LA RAMA

In [ ]:
comando = 'git log --oneline {nombre_rama_principal}'.format(nombre_rama_principal = nombre_rama_principal)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [ ]:
comando = 'git log -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS USANDO **SHOW**

In [ ]:
comando = 'git show -2'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### MOSTRAR LOS METADATOS Y LOS CAMBIOS DE UN COMMIT ESPECIFICO

In [ ]:
# solo en este paso, al hacer el primer commit se crea la rama
comando = 'git show fd01bde637f8bacb4d2a99f6ace425480be3599a'
ejecutar_comando_git(comando)
        
comando = 'git status'
ejecutar_comando_git(comando)

### VER HISTORIAL RESUMIDO DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [ ]:
comando = 'git log --oneline -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS ANTES DE UNA FECHA ESPECIFICA

In [ ]:
comando = 'git log --oneline --before={2023-03-28}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS DESPUES DE UNA FECHA ESPECIFICA

In [ ]:
comando = 'git log --oneline --after={2023-03-27}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE QUIEN HA MANIPULADO UN ARCHIVO ESPECIFICO

In [ ]:
comando = 'git blame requirements.txt'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO

In [ ]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO MAS DETALLADO

In [ ]:
comando = 'git log --decorate --graph --oneline --all'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> BUSCAR FRASE DENTRO DE TODOS LOS ARCHIVOS DE LA RAMA DEL REPOSITORIO</center>

### BUSQUEDA SENSIBLE AL CASO

In [ ]:
frase_a_buscar = '7'

# la busqueda es sensible al caso
comando = 'git grep {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### BUSQUEDA NO SENSIBLE AL CASO

In [ ]:
# busca mayusculas y minusculas
comando = 'git grep -i {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

### BUSQUEDA POR PALABRA ESPECIFICA

In [ ]:
# busca la palabra sola sin mezcla, que no sea prefijo o sufijo, los simbolos no bloquean la busqueda
comando = 'git grep -iw {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

### BUSQUEDA CON INTERVALO DE LINEAS POR ENCIMA Y POR DEBAJO DEL TEXTO A BUSCAR

In [ ]:
# divide el texto y muestra con C cuantas lineas detexto por debajo y por encima existe al filtrar la palabra deseada
comando = 'git grep -i -C 2 {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

### MOSTRAR EL ARCHIVO Y EL NUMERO DE LA LINEA DONDE ESTA EL TEXTO BUSCADO

In [ ]:
comando = 'git grep -i -n {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> MODIFICAR EL MENSAJE DEL ULTIMO COMMIT</center>

In [ ]:
nombre_rama = 'amend_branch'
crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=False)

# EN LA CONSOLA DE VSCODE
# git commit --amend

## SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar el emnsaje del commit anterior por el que se quiere y cerrar el archivo
## validar en local con git log, no afecta la hora del commit original con la fecha del mensaje actualizado
## subir los cambios al repositorio¶
# git push --force origin amend_branch

In [ ]:
comando = 'git log --oneline'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO REMOTO Y LOCAL

In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> REGRESAR A UNA VERSION ANTERIOR DE UN REPOSITORIO GIT</center>

### DETACHED - MOVERSE A UN COMMIT ESPECIFICO Y PONER ESTE COMMIT CON LA NUEVA HEAD

In [ ]:
comando = 'git checkout {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

comando = 'git checkout 6bfbfd4a81082e9312caf0a94ba88d975f9db1e9'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### RETORNA AL HEAD ORIGINAL DE LA RAMA

In [ ]:
comando = 'git switch -'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> RESET COMMITs DE UN REPOSITORIO <br> ELIMINA LOS COMMITS Y AHORA EL NUEVO HEAD ES EL COMMIT A DONDE SE QUIERE IR</br></center>

In [ ]:
nombre_rama = 'reset_branch'
crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=True)

### SOFT RESET n COMMITs **HEAD~n** MANTENIENDO LOS CAMBIOS QUE SE HABIAN HECHO EN LOS ARCHIVOS LOCALES
<center>
    <img width="75%" align="center" src="reset_soft.gif">
</center>
<br>
<center>
    <img width="75%" align="center" src="reflog_reset.gif">
</center>

In [ ]:
# MANITIENE todos los archivos o cambios hechos despues de este commit Y PUEDEN SER GUARDADOS DE NUEVO
comando = 'git reset --soft HEAD~3'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

### HARD RESET n COMMITs **HEAD~n** ELIMINANDO LOS ARCHIVOS Y CAMBIOS QUE SE HABIAN HECHO HASTA ESTE COMMIT
<center>
    <img width="75%" align="center" src="reset_hard.gif">
</center>

### RESET COMMITs HASTA n COMMITS ANTERIORES

In [ ]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git reset --hard HEAD~7'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

### RESET COMMITs ELIMINANDO LOS ARCHIVOS Y CAMBIOS HASTA UN COMMIT ESPECIFICO USANDO EL ID DEL COMMIT

In [ ]:
comando = 'git reset --hard ae943f2' # configuraion general y de ambiente
ejecutar_comando_git(comando)


comando = 'git log --oneline'
ejecutar_comando_git(comando)

## RESET HASTA UN COMMIT ESPECIFICO EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
comando = 'git reset --hard 474faa1' # adicionado archivo pdf
ejecutar_comando_git(comando)

#### CONFIRMAR LOS CAMBIOS DEL RESET EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
# confirmar los cambios al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> REVERTIR COMMITs DE UN REPOSITORIO <br> CREA UN NUEVO COMMIT MANTENIENDO LOS COMMITS ANTERIORES</br></center>
<center>
    <img width="75%" align="center" src="revert.gif">
</center>

In [ ]:
nombre_rama = 'revert_branch'
crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=True)

## REVERT HASTA UN COMMIT ESPECIFICO O UN HEAD~ EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git revert HEAD~24'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

#### CONFIRMAR LOS CAMBIOS DEL REVERT EN EL REPOSITORIO REMOTO

In [ ]:
# confirmar al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> REBASE *JUNTAR VARIOS COMMITS EN UNO SOLO, CAMBIAR EL ORDEN DE LOS COMMITS <br>REESCRIBIENDO EL HISTORIAL Y LOS MENSAJES DE MULTIPLES* COMMITs DE UN REPOSITORIO</br> <br>https://github.com/JorgeCardona/comandos_git</br></center>
<center>
    <img width="75%" align="center" src="rebase.gif">
</center>

In [ ]:
nombre_rama = 'rebase_branch'
crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=True)

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO <br>https://github.com/JorgeCardona/comandos_git</br>

<center>
    <img width="75%" align="center" src="rebase_interactivo.gif">
</center>
<br>
<center>
    <img width="75%" align="center" src="rebase_tipos.png">
</center>
<br>
<center>
    <img width="75%" align="center" src="rebase_squash.gif">
</center>

### n COMMITS ATRAS
# git rebase -i HEAD~9

### UN COMMIT ESPECIFICO
# git rebase -i d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48


##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar la palabra **pick** por **squash** GUARDAR y CERRAR
### DEJAR EL PRIMERO COMO PICK YA QUE SINO, NO EXISTEN COMMITS PARA HACER, GENERA ERROR.
##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar los comentarios por el comentario que se quiere
## subir los cambios al repositorio
# git push --force origin rebase_branch

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO

In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> MERGE RAMAS DE UN REPOSITORIO <br>https://github.com/JorgeCardona/comandos_git</br></center>

<center>
    <img width="75%" align="center" src="merge.gif">
</center>

In [ ]:
nombre_ramas = ['branch_uno','branch_dos']

for nombre_rama in nombre_ramas:
    crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=False)

# <center> UNIR DOS RAMAS LOCALES </center>

<center>
    <img width="75%" align="center" src="merge_conflictos.gif">
</center>

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO
# git merge branch_dos branch_uno

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

# <center> UNIR CAMBIOS DE LA RAMA REMOTA A LA RAMA LOCAL </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
<center>
    <img width="75%" align="center" src="pull.gif">
</center>

# git switch branch_uno
# git pull origin branch_dos

# <center> UNIR CAMBIOS DE LA RAMA LOCAL A LA RAMA REMOTA </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
### hacer un git commit y despues un git pull

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

# <center> VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO</center>

In [ ]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO MAS DETALLADO

In [ ]:
comando = 'git log --decorate --graph --oneline --all'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE QUIEN HA MANIPULADO UN ARCHIVO ESPECIFICO

In [ ]:
comando = 'git blame info.txt'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

#### ELIMINAR LAS RAMAS EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
comando = 'git checkout {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    
    eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> CHERRY PICK, UNIR EL CONTENIDO DE UN COMMIT ESPECIFICO Y QUEDA COMO HEAD <br>https://github.com/JorgeCardona/comandos_git</br></center>

<center>
    <img width="75%" align="center" src="cherry-pick.gif">
</center>

### CREA LAS RAMAS

In [ ]:
nombre_ramas = ['hotfix_branch','cherry_pick_branch']

todos_los_archivos=True
crear_rama_local_y_remota(nombre_rama=nombre_ramas[0], todos_los_archivos=todos_los_archivos)

comando = 'git switch --orphan {nombre_rama}'.format(nombre_rama=nombre_ramas[1])
ejecutar_comando_git(comando)

generar_archivos(todos_los_archivos=False)

comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git commit -m "archivo info.txt para usar con el hotfix_branch" '
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

### PUBLICAR RAMA <br>https://github.com/JorgeCardona/comandos_git</br> </center>


In [ ]:
comando = 'git push --set-upstream origin {nombre_rama}'.format(nombre_rama=nombre_ramas[1])
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

## VER EL LOG DE LA RAMA A USAR COMO BASE PARA EL CHERRY_PICK

In [ ]:
comando = f'git log --oneline {nombre_ramas[0]}'
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[1]}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## CHERRY PICK A UN COMMIT DE LA RAMA **CHERRY_PICK** PARA UNIRLA CON LA RAMA ACTUAL **HOTFIX**

### ADICIONA UN SOLO COMMIT

In [ ]:
comando = 'git status'
ejecutar_comando_git(comando)

unir_commits = 'fd01bde' # archivo README
comando = f'git cherry-pick {unir_commits}' 
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[1]}'
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[0]}'
ejecutar_comando_git(comando)

### ADICIONA VARIOS COMMIT SELECCIONADOS

In [ ]:
comando = 'git status'
ejecutar_comando_git(comando)

unir_commits = 'd481740 6bfbfd4' # archivo requirements.txt, 6bfbfd4
comando = f'git cherry-pick {unir_commits}' 
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[1]}'
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[0]}'
ejecutar_comando_git(comando)

### ADICIONA UN RANGO DE COMMITS
#### SI EXISTE ARCHIVOS CON EL MISMO NOMBRE EN EL INTERVALO DE COMMITS **CREARAN CONFLICTOS**

### SE EXCLUYE EL COMMIT DE INICIO

In [ ]:
commit_inicio = 'c4f3a74' # commit de archivo Dockerfile
commit_final = 'a3af718' # commit de archivo imagen.png

unir_commits = f'{commit_inicio}..{commit_final}' 
comando = f'git cherry-pick {unir_commits}' 
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[1]}'
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[0]}'
ejecutar_comando_git(comando)

### SE INCLUYE EL COMMIT DE INICIO ~

In [ ]:
commit_inicio = '1e9d747' # commit de archivo main.py
commit_final = '7a4b442' # commit de archivo query.sql

unir_commits = f'{commit_inicio}~..{commit_final}'
comando = f'git cherry-pick {unir_commits}' 
ejecutar_comando_git(comando)

comando = f'git log --oneline {nombre_ramas[1]}'
ejecutar_comando_git(comando)

### SUBIR LOS COMMITS ADICIONADOS CON CHERRY-PICK

In [ ]:
comando = 'git push origin'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
# si queda algo pendiente con el cherry-pick
comando = 'git cherry-pick --quit'
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> TAGs <br>https://github.com/JorgeCardona/comandos_git</br></center>

In [ ]:
nombre_rama = 'tag_branch'

crear_rama_local_y_remota(nombre_rama=nombre_rama, todos_los_archivos=True)

### CREAR UN TAG

In [ ]:
nombre_tag = 'v0.0.0'
comando = 'git tag  {nombre_tag} -m "versión Inicial"'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs LOCALES

In [ ]:
comando = 'git tag'
ejecutar_comando_git(comando)

### ELIMINAR UN TAG LOCAL

In [ ]:
comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### CREAR MULTIPLES TAGs

In [ ]:
lista_ramas_locales = obtener_lista_nombres_ramas_git()

for indice, rama in enumerate(lista_ramas_locales):
 
    #cambia de rama para tener el contenido especifico de esa rama en el TAG
    comando = 'git checkout {rama}"'.format(rama=rama)
    ejecutar_comando_git(comando)
    
    # genera el nombre que tendra el TAG
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = 'git tag {nombre_tag} -m "version {indice} del TAG"'.format(nombre_tag=nombre_tag, indice=indice)
    ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs Y VER SU DESCRIPCION

In [ ]:
comando = 'git tag -n'
ejecutar_comando_git(comando)

### PUBLICAR UN TAG ESPECIFICO A UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
for indice, rama in enumerate([lista_ramas_locales[0]]):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = comando = 'git push origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

### LISTAR LOS TAGs REMOTOS

In [ ]:
comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### PUBLICAR TODOS LOS TAGs NO PUBLICADOS EN UN SOLO PUSH AL REPOSITORIO REMOTO

In [ ]:
comando = 'git push --tags'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### ELIMINAR LOS TAGs LOCALES Y REMOTOS

In [ ]:
for indice, rama in enumerate(lista_ramas_locales):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)

    comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

    comando = 'git push --delete origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)
    
comando = 'git tag'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO


In [ ]:
eliminar_rama_local_y_remota(nombre_rama=nombre_rama)

# <center> VALIDAR COMMITS QUE NO FUNCIONAN CON BISECT <br> EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO</br></center>

In [ ]:
nombre_rama_remota_bisect = 'bisect_branch'

comando = 'git checkout -t origin/{nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota_bisect)
ejecutar_comando_git(comando) 
ejecutar_comando_shell()

# git bisect start, git bisect reset, git bisect good, git bisect bad

### INICIALIZO EL ANALISIS CON BISET START

In [ ]:
comando = 'git bisect start'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

### EJECUTO EL CODIGO Y VEO QUE FUNCIONE O FALLE

In [ ]:
def validar_codigo():

    from funciones_matematicas import ejecutar_funciones

    ejecutar_funciones()

In [ ]:
validar_codigo()

# SE EJECUTA EL ULTIMO COMMIT, QUE ENTRE LO NORMAL DEBE FALLAR
## SI FALLA PONGO 'git bisect bad', SI FUNCIONA PONGO 'git bisect good'

In [ ]:
comando = 'git bisect bad'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

## SE SELECCIONA UN COMMIT DONDE SABEMOS QUE EL CODIGO FUNCIONA, POR EJEMPLO EL PRIMER COMMIT

In [ ]:
comando = 'git bisect good 00d5b10'
ejecutar_comando_git(comando)

### VALIDAR DE NUEVO SI FUNCIONA O NO, EN LAS 6 REVISIONES QUE QUEDAN, ESTA VEZ ELIGIO EL DE adicionada funcion raiz cuadrada
## SI FALLA PONGO 'git bisect bad', SI FUNCIONA PONGO 'git bisect good'

In [ ]:
validar_codigo()

In [ ]:
comando = 'git bisect bad'
ejecutar_comando_git(comando)

## VALIDAR DE NUEVO SI FUNCIONA O NO
## SI FALLA PONGO 'git bisect bad', SI FUNCIONA PONGO 'git bisect good'

## YA TERMINO ENCONTRO EL COMMIT QUE FALLA
### AHORA TERMINAMOS CON EL RESET

In [ ]:
comando = 'git bisect reset'
ejecutar_comando_git(comando)

### AHORA REVISO EL COMMIT QUE ESTA GENERANDO ERRORES

In [ ]:
comando = 'git show f649a14'
ejecutar_comando_git(comando)

# <center> LISTAR EL HISTORIAL DE TODAS LAS ACCIONES REALIZADAS EN UN REPOSITORIO GIT <br> LISTADO DESDE EL MAS NUEVO HASTA EL PRIMER CAMBIO</br> <br> PERMITE RECUPERAR COMMITS Y RAMAS BORRADAS ACCIDENTALMENTE USANDO EL HASH DEL COMMIT</br></center>

### OBTENER EL HISTORIAL DE LOS CAMBIOS EN EL REPOSITORIO

In [ ]:
comando = 'git reflog'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## RECUPERAR RAMA ELIMINADA

#### INSPECCIONAMOS EL CONTENIDO DE LA RAMA A RECUPERAR CON LA AYUDA DEL HASH DEL COMMIT

In [ ]:
# adicionado archivo INFO.txt PARA USAR EN REFLOG
commit_a_recuperar = 'e6184ca'
comando = 'git show {commit_a_recuperar}'.format(commit_a_recuperar=commit_a_recuperar)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

#### OBTENEMOS EL CONTENIDO DEL COMMIT

In [ ]:
comando = 'git checkout {commit_a_recuperar}'.format(commit_a_recuperar=commit_a_recuperar)
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

#### CREAMOS UNA NUEVA RAMA A PARTIR EL CONTENIDO DEL COMMIT RECUPERADO

In [ ]:
nombre_rama = 'utils_recuperada'
comando = 'git checkout -b {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

## ELIMINAR RAMA

In [ ]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout  {nombre_rama_principal}'.format(nombre_rama_principal=nombre_rama_principal)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

### OBTENER EL HISTORIAL DE LOS CAMBIOS EN EL REPOSITORIO Y MOSTRAR LOS CAMBIOS DE CADA ARCHIVO EN ESOS CAMBIOS

In [ ]:
comando = 'git reflog -g --patch'
ejecutar_comando_git(comando)
ejecutar_comando_shell()